# parseia-json
Esse script transforma os dados do CSV, já filtrado e com apenas os discursos que mencionam a ditadura, e transforma em um formato .json para gerar a visualização de dados.

#### Importar pacotes

In [1]:
import json
import pandas as pd
import pprint as pp

#### Formatar dados ao longo do tempo

In [2]:
def resample_df(df):
    
    '''
    Essa função recebe o dataframe que veio de 'buscador-da-ditadura.ipynb'
    e 
    '''
    
    # Filtra para manter os anos que temos inteiros
    df = df['2001-01-01':'2018-12-31']
    
    # Baseado em: stackoverflow.com/questions/45908146/

    # Cria um índice no intervalo desejado
    t_index = pd.DatetimeIndex(start='2001-01-01', end='2018-12-31', freq='M')

    # Faz o resample e agregação
    df = df.resample('M').agg({
        # Primeiro, contamos quantos discursos foram registrados no mês
        "FILE":'count', 
        # Depois, quantos deles mecionaram a ditadura.
        # Soma funciona porque, quando há uma menção, o valor é 1. Quando não é, o valor é zero
        "MENTIONS_KEYWORDS":'sum'
    })

    # Aplica no novo índice
    df = df.reindex(t_index)

    # Reseta índice
    df = df.reset_index()

    # Renomeia colunas
    df = df.rename( columns = {
        'index' : "SESSION_DATE",
        'FILE'  : 'ALL_MENTIONS'
    })

    # Preenche os NaNs com zero
    df = df.fillna(0)
    
    return df

#### Parsear json

In [3]:
def parse_json(df, df_t):

    '''
    PARÂMETROS:
    
    df -> O dataframe original, com todas as declarações

    df_t -> O dataframe que já passou pelo resample,
    que é feito na função resample_df(()
    '''
    
    array = [ ]

    for index, row in df_t.iterrows():

        # Seleciona as votações que ocorreram no dia
        this_year  = row.SESSION_DATE.year
        this_month = row.SESSION_DATE.month
        sliced_df = df[ f"{this_year}-{this_month}" ]

        # Filta para manter apenas aquelas em que houve uso de palavra-chave
        sliced_df = sliced_df[ sliced_df.MENTIONS_KEYWORDS == 1 ]

        # Reordena de mais recente para mais tardia
        sliced_df = sliced_df.sort_index( ascending = False ).reset_index()

        obj = { }

        obj["year"]       = int(row.SESSION_DATE.year)
        obj["month"]      = int(row.SESSION_DATE.month)
        obj["total"]      = int(row.ALL_MENTIONS)
        obj["highlights"] = int(row.MENTIONS_KEYWORDS)
        obj["speeches"]   = [ ]

        for i in range(obj["highlights"]):
            inner_obj = { }
            inner_obj["excerpt"] = sliced_df.loc[i, 'FULL_ORIGINAL_QUOTE'][:277] + '...'
            inner_obj["session"] = sliced_df.loc[i, 'SESSION_TYPE']

            obj["speeches"].append(inner_obj)

        array.append(obj)
        
    return array

#### Executar operações

In [4]:
def run_script():
    
    path = "../data/csvs/discursos_jair_bolsonaro_classificados.csv"
    df = pd.read_csv(path)
    
     # Remove dados de discursos em comissão
    df = df[ df.SESSION_TYPE != 'COMISSÃO ESPECIAL (FORA DO PLENÁRIO)' ]
    
    # Passa a data do pronunciamento para o index
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE)
    df = df.set_index("SESSION_DATE")
    
    # Faz o resample temporal
    df_t = resample_df(df)

    array = parse_json(df, df_t)
    
    with open('../data/jsons/data.json', 'w') as fp:
        json.dump(array, fp, indent = 2)
        
    pp.pprint(array)

In [5]:
run_script()

[{'highlights': 0, 'month': 1, 'speeches': [], 'total': 1, 'year': 2001},
 {'highlights': 0, 'month': 2, 'speeches': [], 'total': 4, 'year': 2001},
 {'highlights': 1,
  'month': 3,
  'speeches': [{'excerpt': '– Sr. Presidente, Sras. e Srs. Deputados, depois '
                           'da manifestação do nobre companheiro, Deputado '
                           'Alceu Collares, fico orgulhoso. Acho que até a '
                           'transição para a democracia devia ser feita com a '
                           'volta dos militares ao poder, porque não tivemos '
                           'ditadura aqui. O Brasil cresceu nes...',
                'session': 'COMUNICACOES PARLAMENTARES'}],
  'total': 5,
  'year': 2001},
 {'highlights': 2,
  'month': 4,
  'speeches': [{'excerpt': 'Sr. Presidente, é uma satisfação ocupar a tribuna '
                           'estando V.Exa. na condução da Mesa.Sr. Presidente, '
                           'venho à tribuna para cumprimentar Fernando '
 

                           'minutos, porque hoje é um dia muito especial para '
                           'mim. Sou Capitão do Exército Brasileiro e quero '
                           'falar um pouco sobre o dia 31 de março de 1964. '
                           'Sr. Presidente, Sras. e Srs. Deputados, a data de '
                           'hoje deveria ser comemorada com muita...',
                'session': 'GRANDE EXPEDIENTE'}],
  'total': 6,
  'year': 2005},
 {'highlights': 0, 'month': 4, 'speeches': [], 'total': 2, 'year': 2005},
 {'highlights': 0, 'month': 5, 'speeches': [], 'total': 4, 'year': 2005},
 {'highlights': 2,
  'month': 6,
  'speeches': [{'excerpt': 'Sr. Presidente, hoje, às 15h, haverá sessão solene '
                           'em homenagem aos militares mortos no Araguaia, com '
                           'a presença do coronel Lício Maciel, autor da '
                           'prisão de José Genoíno no passado. Convido todos a '
                           'dela 

                           '"talvez", do "quem sabe?" e disse que queria a '
                           'instalação da Comissão da Verdade para apurar, '
                           'segundo ele, a história da ditadura militar. Quero '
                           'responder a esse Parlamentar que é bo...',
                'session': 'GRANDE EXPEDIENTE'},
               {'excerpt': 'Sr. Presidente, nesse grande embate sobre tarifa '
                           'da Usina Hidrelétrica de Itaipu, quero saudar '
                           'S.Exa., o Presidente da República, General Emílio '
                           'Garrastazu Médici, que foi quem construiu Itaipu! '
                           'E ninguém falou nada aqui. Com a construção dessa '
                           'usina, uma das maiores do mundo,...',
                'session': 'ORDEM DO DIA'}],
  'total': 5,
  'year': 2011},
 {'highlights': 4,
  'month': 5,
  'speeches': [{'excerpt': 'Sr. Presidente, desde dezembro de 2001 encontra

  'speeches': [{'excerpt': 'Sr. Presidente, queria falar 3 minutos, mas serei '
                           'bastante breve. Quero me referir ao Deputado Vitor '
                           'Valim, do PMDB do Ceará. No dia 1º de novembro de '
                           '2003, houve um dos crimes mais bárbaros cometidos '
                           'em nosso País: uma menina de 16 anos de idade e o '
                           'namorado dela, de 19 anos ...',
                'session': 'BREVES COMUNICACOES'},
               {'excerpt': 'Sr. Presidente, como capitão do Exército e '
                           'devidamente autorizado pelo major Olímpio, quero '
                           'nesta data, 31 de março, saudar os militares das '
                           'Forças Armadas e os integrantes das forças '
                           'auxiliares, em memória do Tenente Alberto Mendes '
                           'Júnior, da nossa então gloriosa Força Públic...',
                'session': 